In [1]:
import pandas as pd

x = pd.read_pickle("/home/Kioedru/code/SSGO/data/finetune/9606/train_data_P.pkl")
display(x)

,Entry,Sequence,annotations,Cross-reference (STRING)
0,Q13356,MGKRQHQKDKMYITCAEYTHFYGGKKPDLPQTNFRRLPFDHCSLSL...,[GO:0000209],9606.ENSP00000334553;
1,Q9P2K6,MSAEEMVQIRLEDRCYPVSKRKLIEQSDYFRALYRSGMREALSQEA...,"[GO:0000209, GO:0043161]",9606.ENSP00000370671;
2,P21675,MGPGCDLLLRTAATITAAAIMSDTDSDEDSAGGGPFSLAGFLFGNI...,"[GO:0000209, GO:0006974, GO:0006468, GO:001062...",9606.ENSP00000406549;
3,O94885,MEDAGAAGPGPEPEPEPEPEPEPAPEPEPEPKPGAGTSEAFSRLWT...,[GO:0000209],9606.ENSP00000356437;
4,Q13049,MAAAAASHLNLDALREVLECPICMESFTEEQLRPKLLHCGHTICRQ...,"[GO:0000209, GO:0016567, GO:0045087, GO:0030335]",9606.ENSP00000408292;
...,...,...,...,...
3192,P53396,MSAKAISEQTGKELLYKFICTTSAIQNRFKYARVTPDTDWARLLQD...,[GO:0006695],9606.ENSP00000253792;
3193,Q9UBM7,MAAKSQPNIPKAKSLDGVTNDRTASQGQWGRAWEVDWFSLASVIFL...,[GO:0006695],9606.ENSP00000347717;
3194,P11413,MAEQVALSRTQVCGILREELFQGDAFHQSDTHIFIIMGASGDLAKK...,[GO:0006695],9606.ENSP00000377192;
3195,Q15125,MTTNAGPLHPYWPQHLRLDNFVPNDRPTWHILAGLFSVTGVLVVTT...,[GO:0006695],9606.ENSP00000417052;


In [2]:
import sys

sys.path.append("/home/Kioedru/code/SSGO")
from read_finetune_data import read_gan_seq_embed_and_labels

embs, labels = read_gan_seq_embed_and_labels("P", "9606", "esm2-480")
display(embs.shape, labels.shape)

(5555, 480)

(5555, 45)

In [8]:
import torch


src = torch.rand(3, 32, 512)

In [9]:
src.shape

torch.Size([3, 32, 512])

In [24]:
seq_src = src[2].unsqueeze(0)  # [1,32,512]
ppi_feature_src = src[:2]  # [2,32,512]
ppi_feature_demo = torch.einsum("LBD->BDL", ppi_feature_src).reshape(
    1, 32, -1
)  # [1,32,1024]

seq_src = torch.einsum("LBD->BLD", seq_src)  # [32,1,512]
ppi_feature_src = torch.einsum("LBD->BLD", ppi_feature_src)  # [32,1,1024]
display(seq_src.shape, ppi_feature_src.shape)

torch.Size([32, 1, 512])

torch.Size([32, 2, 512])

In [31]:
src = torch.rand(32, 1, 1536)
src_demo = src.reshape(32, 1, 3, 512)

# 调整维度顺序，得到形状为 [3, 32, 512] 的张量
src_demo = src_demo.permute(2, 0, 3, 1).reshape(3, 32, 512)
display(src_demo.shape)

torch.Size([3, 32, 512])

In [19]:
seq_src = src[2].unsqueeze(0)
ppi_feature_src = src[:2]
display(seq_src.shape, ppi_feature_src.shape)

torch.Size([1, 32, 512])

torch.Size([2, 32, 512])

In [23]:
ppi_demo = torch.einsum("LBD->BDL", ppi_feature_src).reshape(1, 32, -1)
display(ppi_demo.shape)

torch.Size([1, 32, 1024])

In [11]:
seq_src = torch.einsum("LBC->BLC", seq_src)
ppi_feature_src = torch.einsum("LBC->BLC", ppi_feature_src)
display(seq_src.shape, ppi_feature_src.shape)

torch.Size([32, 1, 512])

torch.Size([32, 2, 512])

In [4]:
import torch

src = torch.rand(2, 32, 512)
src = torch.einsum("LBD->BLD", src)
src = src.flatten(1)
src.shape

torch.Size([32, 1024])

In [5]:
import pandas as pd
import numpy as np
import sys
from evaluate_performance import evaluate_performance

In [12]:
predict = pd.read_pickle("/home/Kioedru/code/PO2GO/data_cp/cco_po2go_predictions.pkl")
display(predict)

,labels,preds
0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00010028381, 0.0017451639, 3.7740967e-05, 0..."
1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.004814617, 0.0030973386, 0.0012652369, 0.03..."
2,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.0550805e-09, 0.09757977, 0.27638626, 0.0093..."
3,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.316439e-06, 0.2925879, 0.9794561, 0.0010405..."
4,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0032093953, 0.0043052156, 0.8494466, 0.0034..."
...,...,...
114,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0005686659, 0.0071625984, 0.0016634872, 0.0..."
115,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.362545e-06, 0.01237567, 0.0013884769, 0.006..."
116,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0010194334, 0.1909066, 0.3216787, 0.0298141..."
117,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.2446227e-06, 0.0036227214, 2.7930357e-06, 0..."


In [13]:
labels_matrix = np.vstack(predict["labels"].values)
preds_matrix = np.vstack(predict["preds"].values)

print(labels_matrix.shape)  # 输出 (182, 45)
print(preds_matrix.shape)  # 输出 (182, 45)

(119, 35)
(119, 35)


In [14]:
metric = evaluate_performance(labels_matrix, preds_matrix, preds_matrix > 0.5)

number of test labels is:  35


/home/Kioedru/code/SSGO/codespace/utils/evaluate_performance.py:74: RuntimeWarning: invalid value encountered in divide
  f = np.divide(2 * x * y, (x + y))


In [15]:
display(metric)

{'M-aupr': 0.13534190697259832,
 'M-aupr-labels': [0.010416666666666666,
  0.017918529161345252,
  0.4987931034482759,
  0.010638297872340425,
  0.0,
  0.012828830033886214,
  0.4824721982528809,
  0.1625132623448764,
  0.2875,
  0.041666666666666664,
  0.06249377864438513,
  0.03125,
  0.02631578947368421,
  0.010869565217391304,
  0.1,
  0.13800665456745312,
  0.2002610693400167,
  0.125,
  0.029411764705882353,
  0.08943236496220197,
  0.16666666666666666,
  0.26588372268149885,
  0.25,
  0.08630952380952381,
  0.3658123693686887,
  0.008771929824561403,
  0.0,
  0.7685082621487928,
  0.01888888888888889,
  0.0698005698005698,
  0.26269841269841265,
  0.07142857142857142,
  0.014636282945505369,
  0.03125,
  0.018523002421307506],
 'm-aupr': 0.222587515297305,
 'acc': 0.21008403361344538,
 'F1': 0.23456790123456786,
 'Fmax': 0.3157894736842105}

In [2]:
import torch

display(torch.arange(0, 10))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
import pandas as pd
import torch

tensor1 = pd.read_pickle("/home/Kioedru/code/SSGO/data/compare/ppi_feature_src0.pkl")
tensor2 = pd.read_pickle("/home/Kioedru/code/SSGO/data/compare/ppi_feature_src.pkl")

are_tensors_equal = torch.equal(torch.tensor(tensor1[1]), torch.tensor(tensor2[1]))
print("Are the tensors equal?", are_tensors_equal)

Are the tensors equal? False


/tmp/ipykernel_229032/3508443612.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  are_tensors_equal = torch.equal(torch.tensor(tensor1[1]), torch.tensor(tensor2[1]))
